## Suport Vector Machine with Titanic Dataset

Titanic Dataset contains data for 891 of the real Titanic passengers. Each row represents a single person. The columns describe different attributes about the person including whether they survived, their age, their passenger-class, their sex, the fare they paid etc.

In this notebook we will use SVM classification model to predict who survived the disaster. 

### About the alogrithm

Classifying data is a common task in machine learning. Suppose some given data points each belong to one of two classes, and the goal is to decide which class a new data point will be in. In the case of support-vector machines, a data point is viewed as a *p*-dimensional vector (a list of *p* numbers), and we want to know whether we can separate such points with a *(p-1)*-dimensional hyperplane. 

( Wikipedia: https://en.wikipedia.org/wiki/Support-vector_machine)

Support Vector Machine (SVM) is a supervised machine learning algorithm which can be used for both classification or regression challenges. Support Vectors are simply the co-ordinates of individual observation.

For more information, check out here : https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72

In [32]:
import os
path = '/rapids/notebooks/ml_tutorial/ML_intro/data/'
if not os.path.exists(str(path + 'titanic')):
    print("error, data is missing!")

## 1. Loading the Data

In [33]:
import cudf
titanic_gdf=  cudf.read_csv(str(path + 'titanic/train.csv')) #891x12

## 2. Cleaning the Data 

In [34]:
#encoding
survived = titanic_gdf['Survived'] == 1
die = titanic_gdf['Survived'] == 0


In [35]:
import numpy as np
#encoding
sex = np.zeros(len(titanic_gdf))
sex[titanic_gdf['Sex']== 'male'] = 1
sex[titanic_gdf['Sex']== 'female'] = 0
titanic_gdf['Sex'] = sex

In [36]:
#Fill the NA values with the most common one 
titanic_gdf['Embarked'] = titanic_gdf['Embarked'].fillna('S')
#encoding
embarked = np.zeros(len(titanic_gdf))
embarked[titanic_gdf['Embarked']== 'S'] = 1
embarked[titanic_gdf['Embarked']== 'C'] = 2
embarked[titanic_gdf['Embarked']== 'Q'] = 3
titanic_gdf['Embarked'] = embarked

In [37]:
# Dealing with the missing values in the Age feature.

titanic_gdf['Age'].fillna(titanic_gdf['Age'].mean(),inplace=True)
age = np.zeros(len(titanic_gdf))
#encoding
age[titanic_gdf['Age']<20] = 1
age[(titanic_gdf['Age']>=20)&(titanic_gdf['Age']<60)] = 2
age[(titanic_gdf['Age']>=60)] = 3
titanic_gdf['Age'] = age

In [38]:
# Dropping the columns .ipynb_checkpoints/e are not useing for the analysis 
titanic_gdf = titanic_gdf.drop(columns= ['PassengerId','Name','Ticket','Cabin'])

In [39]:
titanic_gdf.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1.0,2.0,1,0,7.2500,1.0
1,1,1,0.0,2.0,1,0,71.2833,2.0
2,1,3,0.0,2.0,0,0,7.9250,1.0
3,1,1,0.0,2.0,1,0,53.1000,1.0
4,0,3,1.0,2.0,0,0,8.0500,1.0


## 3. Splitting the Data into Training and Testing

In [40]:
import cudf
from cuml.preprocessing.model_selection import train_test_split

target = titanic_gdf['Survived']
titanic_gdf = titanic_gdf.drop(['Survived'])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(titanic_gdf, target,
                                                    train_size=0.8, shuffle = False)

## 4. Modelling

### 4.1. Define Parameters

In [42]:
C = 30 #Penalty parameter C of the error term.
tol = 1e-3   # tolerance for stopping criterion.
kernel= 'rbf' # Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a callable.
gamma = 0.01  # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.

### 4.2. Fit the model

In [43]:
import cuml.svm 
cumlSVC = cuml.svm.SVC(C=C,  gamma=gamma, tol=tol, kernel=kernel)
cumlSVC.fit(X_train, y_train)

SVC(handle=<cuml.common.handle.Handle object at 0x7f91a1f2c3f0>, C=30, kernel='rbf', degree=3, gamma=0.01, coef0=0.0, tol=0.001, cache_size=200.0, max_iter=-1, nochange_steps=1000, verbose=False)

### 4.3. Prediction

In [44]:
cuml_pred = cumlSVC.predict(X_test)

### 4.4. Scoring

In [45]:
cuml_accuracy = np.sum(cuml_pred.to_array()==y_test.to_array()) / y_test.shape[0] * 100
print("Accuracy: cumlSVC {}%".format(cuml_accuracy))

Accuracy: cumlSVC 83.79888268156425%
